In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    'C:\\Users\\taouf\\USE_CASE_3\\Photos', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'C:\\Users\\taouf\\USE_CASE_3\\Photos', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Chargez le modèle préexistant EfficientNetV2L
base_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les poids du modèle préexistant
base_model.trainable = False

# Ajout des couches personnalisées
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sortie binaire : reportage (0) ou studio (1)
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10, 
    batch_size=32
)

Found 1119 images belonging to 2 classes.
Found 279 images belonging to 2 classes.


473176280/473176280 [==============================] - 361s 1us/step
Epoch 1/10


 8/35 [=====>........................] - ETA: 5:02 - loss: 0.6915 - accuracy: 0.5352

In [ ]:
# --- Fine-Tuning ---

base_model.trainable = True

# Débloquer seulement certaines couches
for layer in base_model.layers[:15]: 
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Réentraînement avec les couches dégelées
history_finetune = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen,
    batch_size=32
)

In [ ]:
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test accuracy: {test_acc}")

folder_path = 'photos_usecase3/Photos_Test/reportage_test'

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    return 'Reportage' if prediction[0] < 0.5 else 'Studio'

def predict_on_folder(folder_path, model):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            label = predict_image(img_path, model)
            results.append((filename, label))
            print(f"Image: {filename}, Prédiction: {label}")
    return results
